In [1]:
import sys
from elasticsearch.helpers import BulkIndexError
import json
from elasticsearch import helpers,Elasticsearch
from dotenv import load_dotenv
import os
import pandas as pd

In [2]:


load_dotenv()

client = Elasticsearch(
    ["https://localhost:9200"],  # Địa chỉ Elasticsearch
    basic_auth=("elastic", os.getenv('ELASTIC_PASSWORD')),  # Thay your_password bằng ELASTIC_PASSWORD
    # verify_certs=False  # Bỏ kiểm tra SSL (hoặc cung cấp đường dẫn CA nếu cần)
    ca_certs="ca.crt",
    request_timeout=60

)


In [3]:
client.info()

ObjectApiResponse({'name': 'es01', 'cluster_name': 'docker-cluster', 'cluster_uuid': 'KH-rAW4aTLSEJm4gLMx3mg', 'version': {'number': '8.17.2', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': '747663ddda3421467150de0e4301e8d4bc636b0c', 'build_date': '2025-02-05T22:10:57.067596412Z', 'build_snapshot': False, 'lucene_version': '9.12.0', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'})

In [4]:
client.ping()

True

In [5]:
labels = pd.read_csv("data/AIT_alert/labels.csv")
labels

,scenario,attack,start,end
0,russellmitchell,network_scans,1642993260,1642996606
1,russellmitchell,service_scans,1642996606,1642996645
2,russellmitchell,dirb,1642996645,1642996668
3,russellmitchell,wpscan,1642996668,1642996699
4,russellmitchell,webshell,1642996699,1642996762
...,...,...,...,...
74,wilson,cracking,1644232839,1644234460
75,wilson,reverse_shell,1644234460,1644234498
76,wilson,privilege_escalation,1644234498,1644234533
77,wilson,service_stop,1644144435,1644144437


In [8]:
labels = [
    ("network_scans", 1642507140, 1642508220),
    ("service_scans", 1642508220, 1642508267),
    ("wpscan", 1642508267, 1642508310),
    ("dirb", 1642508310, 1642509480),
    ("webshell", 1642509480, 1642509510),
    ("cracking", 1642509510, 1642511630),
    ("reverse_shell", 1642511630, 1642511670),
    ("privilege_escalation", 1642511670, 1642511693),
    ("service_stop", 1642410286, 1642410288),
    ("dnsteal", 1642410288, 1642413600)
]


# Đường dẫn file input và output
input_file = "data/AIT_alert/fox_aminer.json"

def assign_label(input_file, system_name):
    if system_name == "aminer":
        with open(f"data/AIT_alert/{input_file}.json", "r", encoding="utf-8") as f_in, open(f"data/AIT_alert/{input_file}_labeled.json", "w", encoding="utf-8") as f_out:
            for line in f_in:
                data_json = json.loads(line.strip())

                # Lấy timestamp của document
                timestamp = data_json["LogData"]["Timestamps"][0]

                # Gán nhãn phù hợp hoặc "unknown" nếu không khớp
                label_assigned = "benign"
                for label in labels:
                    if label[1] < timestamp <= label[2]:
                        label_assigned = label[0]
                        break

                # Cập nhật nhãn vào JSON
                data_json["Label"] = label_assigned

                # Ghi dòng JSON đã cập nhật vào file output
                f_out.write(json.dumps(data_json) + "\n")

        print(f"Dữ liệu đã được gán nhãn và lưu vào {output_file}")
    # else :
    #     with open(input_file, "r", encoding="utf-8") as f_in, open(output_file, "w", encoding="utf-8") as f_out:
    #         for line in f_in:
    #             data_json = json.loads(line.strip())
    #
    #             # Lấy timestamp của document
    #             timestamp = data_json["timestamps"]
    #
    #             # Gán nhãn phù hợp hoặc "unknown" nếu không khớp
    #             label_assigned = "benign"
    #             for label in labels:
    #                 if label[1] < timestamp <= label[2]:
    #                     label_assigned = label[0]
    #                     break
    #
    #             # Cập nhật nhãn vào JSON
    #             data_json["Label"] = label_assigned
    #
    #             # Ghi dòng JSON đã cập nhật vào file output
    #             f_out.write(json.dumps(data_json) + "\n")
    #
    #     print(f"Dữ liệu đã được gán nhãn và lưu vào {output_file}")
assign_label("fox_aminer","aminer")
# with open(input_file, "r", encoding="utf-8") as f_in, open(output_file, "w", encoding="utf-8") as f_out:
#     for line in f_in:
#         data_json = json.loads(line.strip())
#
#         # Lấy timestamp của document
#         timestamp = data_json["LogData"]["Timestamps"][0]
#
#         # Gán nhãn phù hợp hoặc "unknown" nếu không khớp
#         label_assigned = "benign"
#         for label in labels:
#             if label[1] <= timestamp < label[2]:
#                 label_assigned = label[0]
#                 break
#
#         # Cập nhật nhãn vào JSON
#         data_json["Label"] = label_assigned
#
#         # Ghi dòng JSON đã cập nhật vào file output
#         f_out.write(json.dumps(data_json) + "\n")
#
# print(f"Dữ liệu đã được gán nhãn và lưu vào {output_file}")


Dữ liệu đã được gán nhãn và lưu vào data/AIT_alert/fox_aminer_labeled2.json


In [18]:

resp = client.indices.put_index_template(
    name="aminer-template",
    index_patterns=[
        "aminer-*"
    ],
    template={
        "settings": {
            "number_of_shards": 1,
            "number_of_replicas": 1
        },
        "mappings": {
            "properties": {
                "AnalysisComponent": {
                    "properties": {
                        "AnalysisComponentIdentifier": {"type": "integer"},
                        "AnalysisComponentType": {"type": "keyword"},
                        "AnalysisComponentName": {"type": "text"},
                        "Message": {"type": "text"},
                        "PersistenceFileName": {"type": "keyword"},
                        "TrainingMode": {"type": "boolean"},
                        "AffectedLogAtomPaths": {"type": "keyword"},
                    }
                },
                "LogData": {
                    "properties": {
                        "RawLogData": {"type": "text"},
                        "Timestamps": {"type": "date", "format": "epoch_second"},
                        "DetectionTimestamp": {"type": "date", "format": "epoch_second"},
                        "LogLinesCount": {"type": "integer"},
                        "LogResources": {"type": "keyword"}
                    }
                },
                "AMiner": {
                    "properties": {
                        "ID": {"type": "ip"}  # Nếu ID không phải là IP, đổi sang keyword
                    }
                },
                "Label": {"type": "keyword"}  # Nhãn của dữ liệu
            }
        }
    },
    priority=102,
)
print(resp)

{'acknowledged': True}


In [9]:

resp = client.indices.put_index_template(
    name="wazuh-template",
    index_patterns=["wazuh-*"],  # Index pattern phù hợp
    template={
        "settings": {
            "number_of_shards": 1,
            "number_of_replicas": 1
        },
        "mappings": {
            "properties": {
                "@timestamp": {"type": "date"},  # Trường thời gian bắt buộc
                "predecoder": {
                    "properties": {
                        "hostname": {"type": "keyword"},
                        "program_name": {"type": "keyword"},
                        "timestamp": {"type": "text"}  # Có thể chuyển thành "date" nếu cần
                    }
                },
                "agent": {
                    "properties": {
                        "ip": {"type": "ip"},  # Kiểu IP
                        "name": {"type": "keyword"},
                        "id": {"type": "keyword"}
                    }
                },
                "manager": {
                    "properties": {
                        "name": {"type": "keyword"}
                    }
                },
                "rule": {
                    "properties": {
                        "firedtimes": {"type": "integer"},
                        "mail": {"type": "boolean"},
                        "level": {"type": "integer"},
                        "pci_dss": {"type": "keyword"},  # Mảng các giá trị keyword
                        "tsc": {"type": "keyword"},
                        "description": {"type": "text"},
                        "groups": {"type": "keyword"},
                        "id": {"type": "keyword"},
                        "nist_800_53": {"type": "keyword"},
                        "gpg13": {"type": "keyword"},
                        "gdpr": {"type": "keyword"}
                    }
                },
                "decoder": {
                    "properties": {
                        "name": {"type": "keyword"}
                    }
                },
                "full_log": {"type": "text"},  # Trường văn bản
                "input": {
                    "properties": {
                        "type": {"type": "keyword"}
                    }
                },
                "location": {"type": "keyword"},  # Đường dẫn file log
                "id": {"type": "keyword"}  # ID của log
            }
        }
    },
    priority=102  # Ưu tiên của template
)

print(resp)

{'acknowledged': True}


In [6]:
def import_json_to_es(file_path, index_name):
     with open(file_path, "r", encoding="utf-8") as f:
        count = 0
        for line in f:
            count += 1
            if count == 100:
                break
            doc = json.loads(line.strip())
            client.index(index=index_name, body=doc)
    # actions = []  # Danh sách dữ liệu tạm thời để lưu batch
    # success_count = 0
    # failed_count = 0
    # batch_size = 500  # Số lượng bản ghi mỗi lần đẩy
    #
    # try:
    #     with open(file_path, "r", encoding="utf-8") as f:
    #         for line in f:
    #             try:
    #                 doc = json.loads(line.strip())  # Đọc từng dòng JSON
    #                 actions.append({
    #                     "_index": index_name,
    #                     "_op_type": "create",
    #                     "_source": doc
    #                 })
    #
    #                 # Khi đủ batch_size, đẩy dữ liệu lên Elasticsearch
    #                 if len(actions) >= batch_size:
    #                     success, failed = helpers.bulk(client, actions)
    #                     success_count += success
    #                     failed_count += failed
    #                     print(f"✅ Đẩy {success} bản ghi thành công, {failed} thất bại.")
    #                     actions.clear()  # Xóa batch đã đẩy
    #
    #             except json.JSONDecodeError as e:
    #                 failed_count += 1
    #                 print(f"❌ Lỗi JSON: {e}")

        # Đẩy phần còn lại (nếu có)
        # if actions:
        #     success, failed = helpers.bulk(client, actions)
        #     success_count += success
        #     failed_count += failed
        #     print(f"✅ Đẩy {success} bản ghi cuối cùng thành công, {failed} thất bại.")
        #
        # print(f"🎉 Hoàn thành! Tổng cộng: {success_count} thành công, {failed_count} thất bại.")

    # except Exception as e:
    #     print(f"❌ Lỗi khi đẩy dữ liệu lên Elasticsearch: {str(e)}")





In [7]:
import_json_to_es("data/AIT_alert/fox_aminer_labeled.json", "aminer-fox-test")

In [38]:
import_json_to_es("data/AIT_alert/fox_wazuh.json", "wazuh-fox")
